# Week 8
weekly practice from The Math of Intelligence by Siraj Raval

In [22]:
import numpy as np

In [23]:
# To save hidden layer
import copy

In [24]:
# Input data - binary numbers for each interger from 0 to 256
int_to_binary = {}
binary_dim = 8
max_val = 2**binary_dim # = 256
binary_val = np.unpackbits(np.array([range(max_val)], dtype = np.uint8).T, axis = 1)

In [26]:
print(binary_val)
print (len(binary_val))
print (len(binary_val[0]))

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 1 0]
 ..., 
 [1 1 1 ..., 1 0 1]
 [1 1 1 ..., 1 1 0]
 [1 1 1 ..., 1 1 1]]
256
8


In [27]:
for i in range(max_val):
    int_to_binary[i] = binary_val[i]

In [28]:
print (int_to_binary[0])

[0 0 0 0 0 0 0 0]


In [29]:
# Sigmoid function
def activate(x, deriv = False):
    if(deriv == True):
        return x*(1 - x)
    return 1 / (1 + np.exp(-x))

In [30]:
# Hyperparameters
inputLayerSize = 2
hiddenLayerSize = 16
outputLayerSize = 1

In [31]:
# 3 Weight Values
w1 = 2 * np.random.random((inputLayerSize, hiddenLayerSize)) - 1
w2 = 2 * np.random.random((hiddenLayerSize, outputLayerSize)) - 1
# From current hidden to next hidden
w_h = 2 * np.random.random((hiddenLayerSize, hiddenLayerSize)) - 1

# Initialize updated weights values
w1_update = np.zeros_like(w1)
w2_update = np.zeros_like(w2)
w_h_update = np.zeros_like(w_h)

In [21]:
# Compute the sum of two integers
for j in range(10000):
    # a + b = c (random values)
    a_int = np.random.randint(max_val / 2)
    b_int = np.random.randint(max_val / 2)
    c_int = a_int + b_int
    
    # Get binary values for a, b, c
    a = int_to_binary[a_int]
    b = int_to_binary[b_int]
    c = int_to_binary[c_int]
    
    # Save predicted binary outputs
    d = np.zeros_like(c)
    
    # Initialize Error
    overallError = 0
    
    # Store output gradients & hidden layer values
    output_layer_gradients = list()
    hidden_layer_values = list()
    hidden_layer_values.append(np.zeros(hiddenLayerSize)) 
    
    for position in range(binary_dim):
        # Input - binary values of a & b
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        # Output - the sum c
        y = np.array([[c[binary_dim - position - 1]]]).T
        
        # Calculate the error
        layer_1 = activate(np.dot(X, w1) + np.dot(hidden_layer_values[-1], w_h))
        layer_2 = activate(np.dot(layer_1, w2))
        output_error = y - layer_2
        
        # Save the error gradients at each step as it will be propagated back
        output_layer_gradients.append((output_error) * activate(layer_2, deriv = True))
        
        # Save the sum of error at each binary position
        overallError += np.abs(output_error[0])
        
        # Round 
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # Save the hidden layer
        hidden_layer_values.append(copy.deepcopy(layer_1))
        
    future_layer_1_gradient = np.zeros(hiddenLayerSize)
    
    # Backpropagate
    for position in range(binary_dim):
        X = np.array([[a[position], b[position]]])
        # The last step hidden layer
        layer_1 = hidden_layer_values[-position - 1]
        # The hidden layer before the current layer
        prev_hidden_layer = hidden_layer_values[-position - 2]
        # Errors at output layer
        output_layer_gradient = output_layer_gradients[-position - 1]
        layer_1_gradient = (future_layer_1_gradient.dot(w_h.T) + output_layer_gradient.dot(w2.T)) * activate(layer_1, deriv=True)
        
        # Update weights
        w2_update += np.atleast_2d(layer_1).T.dot(output_layer_gradient)
        w_h_update += np.atleast_2d(prev_hidden_layer).T.dot(layer_1_gradient)
        w1_update += X.T.dot(layer_1_gradient)

        future_layer_1_gradient = layer_1_gradient
        
    # Update the weights
    w1 += w1_update
    w2 += w2_update
    w_h += w_h_update
    
    # Clear the updated weights values
    w1_update *= 0
    w2_update *= 0
    w_h_update *= 0
    
    # Print out the Progress of the RNN
    if (j % 1000 == 0):
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")

Error:[ 4.02439138]
Pred:[1 1 1 1 1 1 1 1]
True:[0 1 0 1 1 0 0 1]
20 + 69 = 255
------------
Error:[ 0.44111426]
Pred:[1 0 0 1 1 1 0 0]
True:[1 0 0 1 1 1 0 0]
117 + 39 = 156
------------
Error:[ 0.24655136]
Pred:[1 0 1 0 1 0 1 1]
True:[1 0 1 0 1 0 1 1]
126 + 45 = 171
------------
Error:[ 0.10152159]
Pred:[0 1 0 1 1 0 0 0]
True:[0 1 0 1 1 0 0 0]
21 + 67 = 88
------------
Error:[ 0.14344125]
Pred:[0 1 0 1 1 0 0 0]
True:[0 1 0 1 1 0 0 0]
29 + 59 = 88
------------
Error:[ 0.07757354]
Pred:[0 0 1 1 0 0 0 0]
True:[0 0 1 1 0 0 0 0]
35 + 13 = 48
------------
Error:[ 0.07424355]
Pred:[1 0 1 0 0 1 0 0]
True:[1 0 1 0 0 1 0 0]
48 + 116 = 164
------------
Error:[ 1.82523196]
Pred:[0 1 1 0 0 0 1 1]
True:[0 1 1 1 0 0 1 1]
110 + 5 = 99
------------
Error:[ 1.15409802]
Pred:[0 0 1 0 1 0 0 1]
True:[1 0 1 0 1 0 0 1]
120 + 49 = 41
------------
Error:[ 0.03784354]
Pred:[0 1 1 1 1 1 0 0]
True:[0 1 1 1 1 1 0 0]
88 + 36 = 124
------------
